# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas
import numpy as np
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters
import logging
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automlethtime'

experiment=Experiment(ws, experiment_name)

In [16]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'd4ad7261-832d-46b2-b093-22156001df5b'
resource_group = 'aml-quickstarts-164073'
workspace_name = 'quick-starts-ws-164073'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ethdata')
ds=dataset.to_pandas_dataframe()

,Column1,Open,High,Low,Close,Volume,Dividends,Stock Splits,row_num,row_weight
0,2021-09-01 00:00:00,3413.356201,3444.100586,3397.403076,3401.603027,0,0,0,0,1.0
1,2021-09-01 01:00:00,3401.612061,3436.218506,3387.408447,3427.627441,315990016,0,0,1,1.0
2,2021-09-01 02:00:00,3421.749512,3444.784668,3417.847412,3423.020020,255442944,0,0,2,1.0
3,2021-09-01 03:00:00,3417.972412,3443.150146,3412.599854,3443.150146,115654656,0,0,3,1.0
4,2021-09-01 04:00:00,3440.952637,3460.789062,3433.136719,3457.959961,292694016,0,0,4,1.0
...,...,...,...,...,...,...,...,...,...,...
1306,2021-10-27 18:00:00,3990.167969,4007.307617,3980.632324,3983.689697,0,0,0,1306,1.0
1307,2021-10-27 19:00:00,3972.544189,4005.208740,3948.058838,4004.789551,647258112,0,0,1307,1.0
1308,2021-10-27 20:00:00,4003.598877,4024.692139,3965.883789,3984.781738,0,0,0,1308,1.0
1309,2021-10-27 21:00:00,3986.961914,4011.909424,3978.768311,3999.894775,25716736,0,0,1309,1.0


Creating train and test sets


In [6]:
train_data = ds.iloc[:1250]
test_data = ds.iloc[-60:]

label =  "Close"
 
test_labels = test_data.pop(label).values

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:

forecasting_parameters = ForecastingParameters(time_column_name=0, 
                                               forecast_horizon=60,
                                              
                                               freq='H',
                                               target_lags='auto'),
                                               #target_rolling_window_size=10)

ConfigException: ConfigException:
	Message: An empty value for argument [time_column_name] is provided.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "An empty value for argument [time_column_name] is provided.",
        "target": "time_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentBlankOrEmpty"
            }
        },
        "reference_code": "51c61deb-adeb-43a2-b7ca-217da12f0b48"
    }
}

In [21]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = {'time_column_name':'Column1', 
                                               'forecast_horizon':60,
                                               'freq':'H',
                                               'target_lags':'auto',
                                               'target_rolling_window_size':10}

automl_config = AutoMLConfig(task='forecasting',
                             #enable_dnn=True,
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,
                             enable_early_stopping=True,
                             training_data=train_data,
                             label_column_name=label,
                             n_cross_validations=5,
                             enable_ensembling=False,
                             verbosity=logging.INFO,
                             **forecasting_parameters)

In [22]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Experiment,Id,Type,Status,Details Page,Docs Page
automlethtime,AutoML_0b90bf69-8431-4129-a9b6-5bd791b646f3,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [24]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

mean_absolute_error 8.214153859149615
root_mean_squared_error 10.676123328263989
r2_score 0.9607029686760511
median_absolute_error 6.732824435190014
normalized_root_mean_squared_log_error 0.005852862366094036
mean_absolute_percentage_error 0.20258726237892194
normalized_median_absolute_error 0.004307439461342715
normalized_mean_absolute_error 0.00525514526853143
explained_variance 0.9613687873193404
normalized_root_mean_squared_error 0.006830232298642727
root_mean_squared_log_error 0.0026368438262035316
spearman_correlation 0.9761861550427794
residuals aml://artifactId/ExperimentRun/dcid.AutoML_0b90bf69-8431-4129-a9b6-5bd791b646f3_15/residuals
predicted_true aml://artifactId/ExperimentRun/dcid.AutoML_0b90bf69-8431-4129-a9b6-5bd791b646f3_15/predicted_true
forecast_validate_table aml://artifactId/ExperimentRun/dcid.AutoML_0b90bf69-8431-4129-a9b6-5bd791b646f3_15/forecast_validate_table
forecast_train_table aml://artifactId/ExperimentRun/dcid.AutoML_0b90bf69-8431-4129-a9b6-5bd791b646f3_15/

In [30]:
#TODO: Save the best model

best_run.register_model(model_path='outputs/model.pkl', model_name='eth_auto',
                        tags={'Training context':'Auto ML'},
                        properties={'normalized_root_mean_squared_error': best_run_metrics['normalized_root_mean_squared_error']})

Model(workspace=Workspace.create(name='quick-starts-ws-164073', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-164073'), name=eth_auto, id=eth_auto:1, version=1, tags={'Training context': 'Auto ML'}, properties={'normalized_root_mean_squared_error': '0.006830232298642727'})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [fitted_model[0]], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

# Load image data
data = [1.25, 1.5]
# Post raw data to scoring URI
res = request.post(url=service[0].scoring_uri, data=data, headers={'R7': np.float()})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)
service.delete()
